In [ ]:
# Start writing code here...

# Exercises for Session 5

Start a blank deepnote notebook and work on the exercises below. You should reuse code that we used in previous session. Be sure to use `Code` cells when you want to execute code and `Markdown` (You can find markdown by creating a new cell with `+Block` and then clicking on `Markdown`) cells when you want to document your steps.

## Create an overview of genome statistics

Create an overview of genome statistics for Aspergillus and Penicillium species from the [antiSMASH databse](https://antismash-db.secondarymetabolites.org).

*  Visit the [browse section of the antismash database](https://antismash-db.secondarymetabolites.org/browse.html)
* Go through the taxonomy to get the Aspergillus and Penicillium species. If you don't know the taxonomy you can look it up on [NCBI](https://www.ncbi.nlm.nih.gov/taxonomy)
  * Download the corresponding antismash files for *three* organisms of your choice by
    * clicking on the organism name in the antismash overview
    * Now that you are in the overview, click on the download icon and select *Download genbank summary file*
    * Rename the file to something that's easy to use in scripts, e.g. the species name
    * upload the file to deepnote
* Create an overview of genome statistics for each organism (does not need to be combined), you choose how to plot it or create a table similar to *Table 1* in [this publication](https://www.pnas.org/content/115/4/E753)

Conditions:
* It's sufficient to only display Genome size in Mbp, number of proteins, number of contigs and number of scaffolds > 2kbps
  * Go through the [pandas tutorial](https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html), especially [Dataframe aggregation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html) and [Series aggregation](https://pandas.pydata.org/docs/reference/api/pandas.Series.agg.html)
  * to calculate the counts you'll need to select the [correct columns/ subset of the dataframe](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html) use pandas functions like `drop_duplicates()` and `agg`

For this task you'll need to use the functions we created to read the CDS data from *Biopython* `records`.

Extra task:
* How can you show the generated statistics for all organisms in the same table/ plot?

## Create an overview of secondary metabolite genes

The files you downloaded from the antiSMASH database not only contain CDS annotation, but also antismash cluster annotations.

* Use the functions extracting the antiSMASH *"product"* from the Biopython `record`.
* For each organism, create a table or a barplot using `seaborn` of the *"products"*.

Extra task:

Extra task:
* How can you show the generated statistics for all organisms in the same table/ plot?

In [ ]:
"""
Write your comments
here
"""

from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd

def get_df(recs):
    assert isinstance(recs, list), "Need a list as input"
    assert isinstance(recs[0], SeqRecord), "The input list should contain SeqRecord objects"
    fdicts = [] # FROM HERE
    for r in recs:
        for feature in r.features:
            if feature.type=="CDS":
                d = dict(feature.qualifiers)
                for k,v in d.items():
                    if len(v)==1:
                        d[k] = v[0]
                d['contig_id'] = r.id
                d['contig_len'] = len(str(r.seq))
                fdicts.append(d)
    dfa = pd.DataFrame(fdicts) 
    return dfa


r_nidulans = list(SeqIO.parse("aspergillus_nidulans.gbk", "genbank"))
df_nidulans = get_df(r_nidulans)
df_nidulans['org'] = "A. nidulans"

r_tubingensis = list(SeqIO.parse("aspergillus_tubingensis.gbk", "genbank"))
df_tubingensis = get_df(r_tubingensis)
df_tubingensis['org'] = "A. tubingensis"

In [ ]:
"""
Write your comments
here
"""
df_all = pd.concat([df_nidulans, df_tubingensis])
df_nidulans['protein_id'].size # get number of protein products
df_tmp = df_nidulans[['contig_id', 'contig_len']].drop_duplicates() # number of unique contigs
df_tmp['contig_len'].sum() # genome size


30200029

In [ ]:
"""
Write your comments
here
"""
df_all['protein_count_by_contig'] = df_all.groupby('contig_id')['protein_id'].transform(pd.Series.nunique)

In [ ]:
"""
Write your comments
here
"""
df_subset = df_all[
    ['org', 'contig_id',
    'contig_len',
    'protein_count_by_contig']
    ].drop_duplicates()

In [ ]:
"""
Write your comments
here
"""
df_subset

,org,contig_id,contig_len,protein_count_by_contig
0,A. nidulans,NT_107008.1,1482181,486
486,A. nidulans,NT_107009.1,2200800,689
1175,A. nidulans,NW_101435.1,8226,5
1180,A. nidulans,NT_107005.1,1394314,445
1625,A. nidulans,NT_107012.1,2569337,818
...,...,...,...,...
9291,A. tubingensis,NW_023336285.1,2136001,755
10046,A. tubingensis,NW_023336286.1,798263,259
10305,A. tubingensis,NW_023336287.1,1448135,460
10765,A. tubingensis,NW_023336288.1,231917,65


In [ ]:
"""
Write your comments
here
"""
df_subset.groupby('org').agg(
    all_proteins=('protein_count_by_contig', sum),
    genome_size=('contig_len' , sum),
    total_contig=('contig_id' , len)
    
)

,all_proteins,genome_size,total_contig
org,,,
A. nidulans,9556,30200029,77
A. tubingensis,11477,35047229,15


In [ ]:
"""
Write your comments
here
"""
def get_bgc_df(records, o):
    results = []
    for r in records:
        for f in r.features:
            if f.type == 'protocluster':
                if f.qualifiers:
                    tmp = f.qualifiers.get('product')
                    if tmp:
                        tmp = tmp[0]
                        results.append({'org': o, 'product': tmp})

    return pd.DataFrame(results)


tubingensis = list(SeqIO.parse(
    "aspergillus_tubingensis.gbk", "genbank")
    )
df_tubingensis = get_bgc_df(tubingensis, 'A. tubingensis')

nidulans = list(SeqIO.parse(
    "aspergillus_nidulans.gbk", "genbank")
    )
df_nidulans = get_bgc_df(nidulans, 'A. nidulans')

bgc_df = pd.concat([df_tubingensis, df_nidulans])

In [ ]:
"""
Write your comments
here
"""
bgc_overview = bgc_df.groupby(['org', 'product']).agg(product_count=('product', len)).reset_index()

bgc_overview

,org,product,product_count
0,A. nidulans,NRPS,13
1,A. nidulans,NRPS-like,11
2,A. nidulans,T1PKS,23
3,A. nidulans,betalactone,3
4,A. nidulans,indole,5
5,A. nidulans,terpene,9
6,A. tubingensis,NRPS,20
7,A. tubingensis,NRPS-like,20
8,A. tubingensis,T1PKS,39
9,A. tubingensis,T3PKS,1


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c9c5a4ed-74b7-4c42-aa37-668590618479' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>